# Looking at output of VASP calculations to analyse accuracy of GGA + correction vs HSE optics spectra

In [2]:
from pymatgen.io.vasp import Vasprun
from vasppy.optics import absorption_coefficient as ac
import numpy as np
import matplotlib.pyplot as plt
from vasprun import vasprun
from pymatgen.electronic_structure.bandstructure import BandStructure
from SLME import Efficiency
import json
import pandas as pd
import re
import sys
import os

In [3]:
def loadjson(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

In [4]:
def dumpjson(data,filename):
    with open(filename, 'w+') as f:
        json.dump(data, f)

In [5]:
def shifter(spectrum,diff):
    if len(spectrum) == 100:
        new_alphas = np.append(np.array([0]),spectrum)
    else:
        new_alphas = spectrum
    alphas = np.array(new_alphas)
    Es = np.linspace(0,5,101)
    shifted_Es = Es+diff
    with_zero = np.insert(shifted_Es,0,0)
    right_length = np.delete(with_zero,-1)
    new_abs=np.interp(Es,right_length,alphas)
    return new_abs

In [6]:
def analyse_band_structure(vasprun_path):
    """
    analyse band structure from VASP vasprun.xml file
    
    Parameters:
    -----------
    vasprun_path : str
        Path to the vasprun.xml file
    
    Returns:
    --------
    dict containing CBM and VBM information
    """
    # Parse the vasprun.xml file
    vasprun = Vasprun(vasprun_path, parse_projected_eigen=True,parse_potcar_file=False)
    
    # Get the band structure
    bs = vasprun.get_band_structure()
    
    # Find the Fermi energy
    fermi_energy = vasprun.efermi
    
    # Find CBM and VBM
    cbm_info = bs.get_cbm()
    vbm_info = bs.get_vbm()
    
    # Extract K-point information
    cbm_kpoint = bs.kpoints[cbm_info['kpoint_index'][0]]
    vbm_kpoint = bs.kpoints[vbm_info['kpoint_index'][0]]
    
    return {
        'CBM': {
            'energy': cbm_info['energy'],
            'k_point': cbm_kpoint.frac_coords,
            'k_point_index': cbm_info['kpoint_index'][0]
        },
        'VBM': {
            'energy': vbm_info['energy'],
            'k_point': vbm_kpoint.frac_coords,
            'k_point_index': vbm_info['kpoint_index'][0]
        },
        'fermi_energy': fermi_energy,
        'band_gap': bs.get_band_gap()['energy']
    }

# Print results
def display_band_results(analysis, name):
    print(f"Band Structure Analysis for {name}:")
    print("\nCBM Information:")
    print(f"Energy: {band_analysis['CBM']['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {band_analysis['CBM']['k_point']}")
    print(f"K-point Index: {band_analysis['CBM']['k_point_index']}")

    print("\nVBM Information:")
    print(f"Energy: {band_analysis['VBM']['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {band_analysis['VBM']['k_point']}")
    print(f"K-point Index: {band_analysis['VBM']['k_point_index']}")
    print(f"\nFermi Energy: {band_analysis['fermi_energy']:.4f} eV")
    print(f"Band Gap: {band_analysis['band_gap']:.4f} eV")

In [7]:


def extract_cpu_time(outcar_path):
    """
    Extract the total CPU time from a VASP OUTCAR file.

    Args:
        outcar_path (str): Path to the OUTCAR file

    Returns:
        float: Total CPU time in seconds, or None if not found
    """
    # Check if file exists
    if not os.path.isfile(outcar_path):
        print(f"Error: File {outcar_path} not found.")
        return None

    # Pattern to match the CPU time
    pattern = r"Total CPU time used \(sec\):\s*(\d+\.\d+)"

    # Read the file from the end since the CPU time is usually at the end
    with open(outcar_path, 'r') as file:
        # Read the last 50 lines which should contain the CPU time
        lines = file.readlines()
        last_lines = lines[-50:] if len(lines) > 50 else lines

        # Join the last lines into a single string
        text = ''.join(last_lines)

        # Search for the pattern
        match = re.search(pattern, text)

        if match:
            cpu_time = float(match.group(1))
            return cpu_time
        else:
            # If not found in the last 50 lines, search the entire file
            file.seek(0)
            text = file.read()
            match = re.search(pattern, text)

            if match:
                cpu_time = float(match.group(1))
                return cpu_time
            else:
                print("CPU time not found in the OUTCAR file.")
                return None

    if len(sys.argv) > 1:
        outcar_path = sys.argv[1]
    else:
        outcar_path = input("Enter the path to the OUTCAR file: ")

    cpu_time = extract_cpu_time(outcar_path)

    if cpu_time is not None:
        print(f"Total CPU time: {cpu_time} seconds")
        # Convert to hours, minutes, seconds for better readability
        hours = int(cpu_time // 3600)
        minutes = int((cpu_time % 3600) // 60)
        seconds = cpu_time % 60
        print(f"Total CPU time: {hours}h {minutes}m {seconds:.2f}s")


In [8]:
eff = Efficiency()

In [9]:
vasprun_path = 'Si/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'Si GGA')

Band Structure Analysis for Si GGA:

CBM Information:
Energy: 6.2345 eV
K-point (Fractional Coordinates): [0.375 0.375 0.   ]
K-point Index: 17

VBM Information:
Energy: 5.7214 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 5.7382 eV
Band Gap: 0.5131 eV


In [10]:
extract_cpu_time('Si/gga/bands/OUTCAR')

4.938

In [11]:
band_analysis = analyse_band_structure('Cu2O/gga/bands/vasprun.xml')
display_band_results(band_analysis,'Cu2O GGA')

Band Structure Analysis for Cu2O GGA:

CBM Information:
Energy: 5.2389 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

VBM Information:
Energy: 4.7131 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 4.7148 eV
Band Gap: 0.5258 eV


In [12]:
def analyse_static_band_structure(vasprun_path, spin_polarised=False):
    """
    analyse band structure from a static VASP calculation
    
    Parameters:
    -----------
    vasprun_path : str
        Path to the vasprun.xml file
    spin_polarised : bool, optional
        Whether the calculation is spin-polarised (default: False)
    
    Returns:
    --------
    dict containing approximate CBM and VBM information
    """
    # Parse the vasprun.xml file
    vasprun = Vasprun(vasprun_path, parse_projected_eigen=True,parse_potcar_file=False)
    
    # Get the Fermi energy
    fermi_energy = vasprun.efermi
    
    # Extract actual k-points
    actual_kpoints = vasprun.actual_kpoints
    
    # Extract eigenvalues
    eigenvalues = vasprun.eigenvalues
    
    # Flatten and process eigenvalues
    all_eigenvalues = []
    all_kpoints = []
    
    if spin_polarised:
        # Handle spin-up and spin-down separately
        for spin, spin_data in eigenvalues.items():
            spin_eigenvalues = []
            spin_kpoints = []
            
            for kpoint_index, kpoint_eigenvalues in enumerate(spin_data):
                # Filter out zero eigenvalues
                valid_eigenvalues = kpoint_eigenvalues[kpoint_eigenvalues != 0]
                
                # Store eigenvalues and corresponding k-points
                spin_eigenvalues.extend(valid_eigenvalues)
                spin_kpoints.extend([actual_kpoints[kpoint_index]] * len(valid_eigenvalues))
            
            # Store spin-specific data
            all_eigenvalues.append(np.array(spin_eigenvalues))
            all_kpoints.append(spin_kpoints)
        
        # Find VBM and CBM for each spin channel
        vbm_info = []
        cbm_info = []
        
        for spin_eigenvalues, spin_kpoints in zip(all_eigenvalues, all_kpoints):
            # Find eigenvalues just below and above Fermi energy
            valence_mask = spin_eigenvalues <= fermi_energy
            conduction_mask = spin_eigenvalues > fermi_energy
            
            # Approximate VBM and CBM
            vbm_energy = np.max(spin_eigenvalues[valence_mask]) if np.any(valence_mask) else None
            cbm_energy = np.min(spin_eigenvalues[conduction_mask]) if np.any(conduction_mask) else None
            
            # Find corresponding k-points
            vbm_kpoint_index = np.where(spin_eigenvalues == vbm_energy)[0][0] if vbm_energy is not None else None
            cbm_kpoint_index = np.where(spin_eigenvalues == cbm_energy)[0][0] if cbm_energy is not None else None
            
            vbm_info.append({
                'energy': vbm_energy,
                'k_point': spin_kpoints[vbm_kpoint_index] if vbm_kpoint_index is not None else None,
                'k_point_index': vbm_kpoint_index
            })
            
            cbm_info.append({
                'energy': cbm_energy,
                'k_point': spin_kpoints[cbm_kpoint_index] if cbm_kpoint_index is not None else None,
                'k_point_index': cbm_kpoint_index
            })
        
        return {
            'VBM': vbm_info,
            'CBM': cbm_info,
            'fermi_energy': fermi_energy,
            'band_gap': [cbm['energy'] - vbm['energy'] for cbm, vbm in zip(cbm_info, vbm_info)] if all(cbm['energy'] is not None and vbm['energy'] is not None for cbm, vbm in zip(cbm_info, vbm_info)) else None
        }
    
    else:
        # Spin-unpolarised calculation
        for spin, spin_data in eigenvalues.items():
            for kpoint_index, kpoint_eigenvalues in enumerate(spin_data):
                # Filter out eigenvalues
                valid_eigenvalues = kpoint_eigenvalues[kpoint_eigenvalues != 0]
                
                # Store eigenvalues and corresponding k-points
                all_eigenvalues.extend(valid_eigenvalues)
                all_kpoints.extend([actual_kpoints[kpoint_index]] * len(valid_eigenvalues))
        
        # Convert to numpy array for easier manipulation
        all_eigenvalues = np.array(all_eigenvalues)
        
        # Find eigenvalues just below and above Fermi energy
        valence_mask = all_eigenvalues <= fermi_energy
        conduction_mask = all_eigenvalues > fermi_energy
        
        # Approximate VBM and CBM
        vbm_energy = np.max(all_eigenvalues[valence_mask])
        cbm_energy = np.min(all_eigenvalues[conduction_mask])
        
        # Find corresponding k-points
        vbm_kpoint_index = np.where(all_eigenvalues == vbm_energy)[0][0]
        cbm_kpoint_index = np.where(all_eigenvalues == cbm_energy)[0][0]
        
        return {
            'VBM': {
                'energy': vbm_energy,
                'k_point': all_kpoints[vbm_kpoint_index],
                'k_point_index': vbm_kpoint_index
            },
            'CBM': {
                'energy': cbm_energy,
                'k_point': all_kpoints[cbm_kpoint_index],
                'k_point_index': cbm_kpoint_index
            },
            'fermi_energy': fermi_energy,
            'band_gap': cbm_energy - vbm_energy
        }

# Example usage
vasprun_path = 'Cu2O/gga/optics/vasprun.xml'

# For spin-unpolarised calculation
print("Spin-Unpolarised Calculation:")
static_band_analysis = analyse_static_band_structure(vasprun_path, spin_polarised=False)
print("\nVBM Information:")
print(f"Energy: {static_band_analysis['VBM']['energy']:.4f} eV")
print(f"K-point (Reciprocal Space Coordinates): {static_band_analysis['VBM']['k_point']}")
print(f"K-point Index: {static_band_analysis['VBM']['k_point_index']}")

print("\nCBM Information:")
print(f"Energy: {static_band_analysis['CBM']['energy']:.4f} eV")
print(f"K-point (Reciprocal Space Coordinates): {static_band_analysis['CBM']['k_point']}")
print(f"K-point Index: {static_band_analysis['CBM']['k_point_index']}")

print(f"\nFermi Energy: {static_band_analysis['fermi_energy']:.4f} eV")
print(f"Approximate Band Gap: {static_band_analysis['band_gap']:.4f} eV")

Spin-Unpolarised Calculation:

VBM Information:
Energy: 4.7131 eV
K-point (Reciprocal Space Coordinates): [0.0, 0.0, 0.0]
K-point Index: 50

CBM Information:
Energy: 5.2389 eV
K-point (Reciprocal Space Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Fermi Energy: 4.7284 eV
Approximate Band Gap: 0.5258 eV


In [13]:
print("\n\nSpin-polarised Calculation:")
vasprun_path = 'Cu2O/gga/static/vasprun.xml'
static_band_analysis_spin = analyse_static_band_structure(vasprun_path, spin_polarised=True)
for spin in [0, 1]:  # Spin-up and spin-down
    print(f"\nSpin Channel {spin}:")
    print("VBM Information:")
    print(f"Energy: {static_band_analysis_spin['VBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['VBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['VBM'][spin]['k_point_index']}")     
    print("\nCBM Information:")
    print(f"Energy: {static_band_analysis_spin['CBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['CBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['CBM'][spin]['k_point_index']}") 
print(f"\nFermi Energy: {static_band_analysis_spin['fermi_energy']:.4f} eV")
print(f"Approximate Band Gaps: {static_band_analysis_spin['band_gap']}")



Spin-polarised Calculation:

Spin Channel 0:
VBM Information:
Energy: 4.7130 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 50

CBM Information:
Energy: 5.2388 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Spin Channel 1:
VBM Information:
Energy: 4.7131 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 50

CBM Information:
Energy: 5.2388 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Fermi Energy: 4.7602 eV
Approximate Band Gaps: [0.5258000000000003, 0.5257000000000005]


In [14]:
print("\n\nSpin-polarised Calculation:")
vasprun_path = 'Cu2O/hse/static/vasprun.xml'
static_band_analysis_spin = analyse_static_band_structure(vasprun_path, spin_polarised=True)
for spin in [0, 1]:  # Spin-up and spin-down
    print(f"\nSpin Channel {spin}:")
    print("VBM Information:")
    print(f"Energy: {static_band_analysis_spin['VBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['VBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['VBM'][spin]['k_point_index']}")     
    print("\nCBM Information:")
    print(f"Energy: {static_band_analysis_spin['CBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['CBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['CBM'][spin]['k_point_index']}") 
print(f"\nFermi Energy: {static_band_analysis_spin['fermi_energy']:.4f} eV")
print(f"Approximate Band Gaps: {static_band_analysis_spin['band_gap']}")



Spin-polarised Calculation:

Spin Channel 0:
VBM Information:
Energy: 3.8641 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 50

CBM Information:
Energy: 5.9305 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Spin Channel 1:
VBM Information:
Energy: 3.8652 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 50

CBM Information:
Energy: 5.9319 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Fermi Energy: 3.8708 eV
Approximate Band Gaps: [2.0664000000000002, 2.0666999999999995]


In [15]:
print("\n\nSpin-polarised Calculation:")
vasprun_path = 'Cu2O/hse/gap/vasprun.xml'
static_band_analysis_spin = analyse_static_band_structure(vasprun_path, spin_polarised=True)
for spin in [0, 1]:  # Spin-up and spin-down
    print(f"\nSpin Channel {spin}:")
    print("VBM Information:")
    print(f"Energy: {static_band_analysis_spin['VBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['VBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['VBM'][spin]['k_point_index']}")     
    print("\nCBM Information:")
    print(f"Energy: {static_band_analysis_spin['CBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['CBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['CBM'][spin]['k_point_index']}") 
print(f"\nFermi Energy: {static_band_analysis_spin['fermi_energy']:.4f} eV")
print(f"Approximate Band Gaps: {static_band_analysis_spin['band_gap']}")



Spin-polarised Calculation:

Spin Channel 0:
VBM Information:
Energy: 4.2681 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 54

CBM Information:
Energy: 6.6422 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Spin Channel 1:
VBM Information:
Energy: 4.2680 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 54

CBM Information:
Energy: 6.6422 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 56

Fermi Energy: 4.3799 eV
Approximate Band Gaps: [2.3741000000000003, 2.3742]


In [16]:
vasprun_path = 'InCuSe2/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'InCuSe2 GGA')

Band Structure Analysis for InCuSe2 GGA:

CBM Information:
Energy: 4.1359 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

VBM Information:
Energy: 4.1264 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 4.1350 eV
Band Gap: 0.0095 eV


In [17]:
vasprun_path = 'MAPbI3/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'MAPI GGA')

Band Structure Analysis for MAPI GGA:

CBM Information:
Energy: 2.7657 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

VBM Information:
Energy: 1.0823 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 1.0858 eV
Band Gap: 1.6834 eV


In [18]:
vasprun_path = 'GaCuSe2/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'GaCuSe2 GGA')

Band Structure Analysis for GaCuSe2 GGA:

CBM Information:
Energy: 4.0091 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

VBM Information:
Energy: 3.8839 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 3.9095 eV
Band Gap: 0.1252 eV


In [19]:
vasprun_path = 'CuSbS2/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'CuSbS2 GGA')

Band Structure Analysis for CuSbS2 GGA:

CBM Information:
Energy: 6.5196 eV
K-point (Fractional Coordinates): [0.   0.25 0.  ]
K-point Index: 4

VBM Information:
Energy: 5.8100 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 5.8211 eV
Band Gap: 0.7096 eV


In [20]:
vasprun_path = 'Sb2S3/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'Sb2S3 GGA')

Band Structure Analysis for Sb2S3 GGA:

CBM Information:
Energy: 5.7058 eV
K-point (Fractional Coordinates): [0.  0.5 0. ]
K-point Index: 4

VBM Information:
Energy: 4.4497 eV
K-point (Fractional Coordinates): [0.  0.  0.5]
K-point Index: 8

Fermi Energy: 4.4726 eV
Band Gap: 1.2561 eV


In [21]:
vasprun_path = 'Sb2Se3/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'Sb2Se3 GGA')

Band Structure Analysis for Sb2Se3 GGA:

CBM Information:
Energy: 4.6873 eV
K-point (Fractional Coordinates): [0.  0.5 0. ]
K-point Index: 4

VBM Information:
Energy: 3.8021 eV
K-point (Fractional Coordinates): [ 0.  -0.   0.5]
K-point Index: 8

Fermi Energy: 3.8264 eV
Band Gap: 0.8852 eV


In [22]:
vasprun_path = 'SnS/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'SnS GGA')

Band Structure Analysis for SnS GGA:

CBM Information:
Energy: 6.3830 eV
K-point (Fractional Coordinates): [0.28571429 0.         0.        ]
K-point Index: 2

VBM Information:
Energy: 5.3336 eV
K-point (Fractional Coordinates): [-0.   0.5  0.5]
K-point Index: 28

Fermi Energy: 5.3490 eV
Band Gap: 1.0494 eV


In [23]:
vasprun_path = 'ZnCu2SnS4/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'ZnCu2SnS4 GGA')

Band Structure Analysis for ZnCu2SnS4 GGA:

CBM Information:
Energy: 5.0454 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

VBM Information:
Energy: 4.8945 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 4.8968 eV
Band Gap: 0.1509 eV


In [24]:
vasprun_path = 'MAPbI3/gga/bands/vasprun.xml'
band_analysis = analyse_band_structure(vasprun_path)
display_band_results(band_analysis,'MAPbI3 GGA')

Band Structure Analysis for MAPbI3 GGA:

CBM Information:
Energy: 2.7657 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

VBM Information:
Energy: 1.0823 eV
K-point (Fractional Coordinates): [0. 0. 0.]
K-point Index: 0

Fermi Energy: 1.0858 eV
Band Gap: 1.6834 eV


In [25]:
print("\n\nSpin-polarised Calculation:")
vasprun_path = 'MAPbI3/hse/gap/vasprun.xml'
static_band_analysis_spin = analyse_static_band_structure(vasprun_path, spin_polarised=True)
for spin in [0, 1]:  # Spin-up and spin-down
    print(f"\nSpin Channel {spin}:")
    print("VBM Information:")
    print(f"Energy: {static_band_analysis_spin['VBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['VBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['VBM'][spin]['k_point_index']}")     
    print("\nCBM Information:")
    print(f"Energy: {static_band_analysis_spin['CBM'][spin]['energy']:.4f} eV")
    print(f"K-point (Fractional Coordinates): {static_band_analysis_spin['CBM'][spin]['k_point']}")
    print(f"K-point Index: {static_band_analysis_spin['CBM'][spin]['k_point_index']}") 
print(f"\nFermi Energy: {static_band_analysis_spin['fermi_energy']:.4f} eV")
print(f"Approximate Band Gaps: {static_band_analysis_spin['band_gap']}")



Spin-polarised Calculation:

Spin Channel 0:
VBM Information:
Energy: 0.6907 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 198

CBM Information:
Energy: 1.0000 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 1

Spin Channel 1:
VBM Information:
Energy: 0.6908 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 198

CBM Information:
Energy: 1.0000 eV
K-point (Fractional Coordinates): [0.0, 0.0, 0.0]
K-point Index: 1

Fermi Energy: 0.7584 eV
Approximate Band Gaps: [0.3093, 0.30920000000000003]


In [26]:
static_band_analysis_spin

{'VBM': [{'energy': 0.6907, 'k_point': [0.0, 0.0, 0.0], 'k_point_index': 198},
  {'energy': 0.6908, 'k_point': [0.0, 0.0, 0.0], 'k_point_index': 198}],
 'CBM': [{'energy': 1.0, 'k_point': [0.0, 0.0, 0.0], 'k_point_index': 1},
  {'energy': 1.0, 'k_point': [0.0, 0.0, 0.0], 'k_point_index': 1}],
 'fermi_energy': 0.75841133,
 'band_gap': [0.3093, 0.30920000000000003]}

So, what do we want for each material?\
Band-gaps from:
- GGA bands
- HSE 1-3k
- HSE static

Timings for:
- GGA bands, static, optics
- HSE static, optics, 1-3k

Comparison between:
- GGA optics + HSE-GGA correction (via HSE static) resultant SLME
- SLME from HSE optics
- Offsets?

In [27]:
def band_data(material,dict):
    gga_bands_path = material+'/gga/bands/vasprun.xml'
    gga_bands_vasprun = Vasprun(gga_bands_path, parse_projected_eigen=True,parse_potcar_file=False)
    dict[material]['GGA gap'] = gga_bands_vasprun.eigenvalue_band_properties[0]

In [28]:
def hse_static(material,dict,spin_polarised=True):
    hse_static_path = material+'/hse/static/vasprun.xml'
    hse_static_vasprun = Vasprun(hse_static_path, parse_projected_eigen=True,parse_potcar_file=False)
    dict[material]['HSE static gap'] = hse_static_vasprun.eigenvalue_band_properties[0]

In [29]:
def hse_few(material,dict,spin_polarised=True):
    hse_few_path = material+'/hse/gap/vasprun.xml'
    hse_few_vasprun = Vasprun(hse_few_path, parse_projected_eigen=True,parse_potcar_file=False)
    dict[material]['HSE few gap'] = hse_few_vasprun.eigenvalue_band_properties[0]

In [30]:
def shifted_optics(material,dict,energies):
    gga_optics_path = material+'/gga/optics/vasprun.xml'
    gga_optics_vasprun = Vasprun(gga_optics_path,parse_potcar_file=False)
    gga_alphas = gga_optics_vasprun.optical_absorption_coeff
    corr = dict[material]['HSE static gap'] - dict[material]['GGA gap']
    gga_energies = gga_optics_vasprun.dielectric[0]
    gga_interp_alphas = np.interp(energies,gga_energies,gga_alphas)
    shifted_alphas = shifter(gga_interp_alphas,corr)
    shifted_SLME = eff.calculate(energies,shifted_alphas)
    dict[material]['GGA + corr SLME'] = shifted_SLME
    return gga_interp_alphas, shifted_alphas

In [31]:
def hse_optics(material,dict,energies):
    hse_optics_path = material+'/hse/optics/vasprun.xml'
    hse_optics_vasprun = Vasprun(hse_optics_path,parse_potcar_file=False)
    hse_alphas = hse_optics_vasprun.optical_absorption_coeff
    hse_energies = hse_optics_vasprun.dielectric[0]
    hse_interp_alphas = np.interp(energies,hse_energies,hse_alphas)
    hse_SLME = eff.calculate(energies,hse_interp_alphas)
    dict[material]['HSE SLME'] = hse_SLME
    return hse_interp_alphas

In [32]:
def gga_bands_time(material,dict):
    gga_bands_path = material+'/gga/bands/OUTCAR'
    time = extract_cpu_time(gga_bands_path)
    dict[material]['GGA bands time'] = time

In [33]:
def gga_static_time(material,dict):
    gga_static_path = material+'/gga/static/OUTCAR'
    time = extract_cpu_time(gga_static_path)
    dict[material]['GGA static time'] = time

In [34]:
def gga_optics_time(material,dict):
    gga_optics_path = material+'/gga/optics/OUTCAR'
    time = extract_cpu_time(gga_optics_path)
    dict[material]['GGA optics time'] = time

In [35]:
def hse_static_time(material,dict):
    hse_static_path = material+'/hse/static/OUTCAR'
    time = extract_cpu_time(hse_static_path)
    dict[material]['HSE static time'] = time

In [36]:
def hse_optics_time(material,dict):
    hse_optics_path = material+'/hse/optics/OUTCAR'
    time = extract_cpu_time(hse_optics_path)
    dict[material]['HSE optics time'] = time

In [37]:
def hse_few_time(material,dict):
    hse_few_path = material+'/hse/gap/OUTCAR'
    time = extract_cpu_time(hse_few_path)
    dict[material]['HSE few time'] = time

In [38]:
def all_fields(material,dict):
    energies = np.linspace(0,5,101)
    try:
        band_data(material,dict)
    except:
        print(f"Error in {material} band data")
    try:
        hse_static(material,dict)
    except:
        print(f"Error in {material} HSE static")
    try:
        hse_few(material,dict)
    except:
        print(f"Error in {material} HSE few")
    try:
        gga_alphas, shifted_alphas = shifted_optics(material,dict,energies)
    except:
        print(f"Error in {material} optics")
    try:
        hse_alphas = hse_optics(material,dict,energies)
    except:
        print(f"Error in {material} HSE optics")
    fig,ax = plt.subplots(figsize=(5,5))
    ax.plot(energies,gga_alphas,label = 'GGA')
    ax.plot(energies,hse_alphas,label = 'HSE')
    ax.plot(energies,shifted_alphas,label = 'Shifted GGA')
    fig.legend()
    ax.set_xlabel('Energy (eV)')
    ax.set_ylabel(r'$\alpha$ / cm$^{-1}$')
    try:
        gga_static_time(material,dict)
    except:
        print(f"Error in {material} GGA static time")
    try:
        gga_bands_time(material,dict)
    except:
        print(f"Error in {material} GGA bands time")
    try:
        gga_optics_time(material,dict)
    except:
        print(f"Error in {material} GGA optics time")
    try:
        hse_static_time(material,dict)
    except:
        print(f"Error in {material} HSE static time")
    try:
        hse_optics_time(material,dict)
    except:
        print(f"Error in {material} HSE optics time")
    try:
        hse_few_time(material,dict)
    except:
        print(f"Error in {material} HSE few time")
    print(f"Following entries altered for {material} in dictionary", dict[material])
    return fig

In [39]:
key_materials = loadjson('key_materials.json')

In [40]:
data = {key: {} for key in key_materials}

In [41]:
fig = all_fields('Si', data)

Following entries altered for Si in dictionary {'GGA gap': 0.5130999999999997, 'HSE static gap': 1.1666999999999996, 'HSE few gap': 1.7303000000000006, 'GGA + corr SLME': 3.1583337550403137, 'HSE SLME': 1.887657797722469, 'GGA static time': 8.323, 'GGA bands time': 4.938, 'GGA optics time': 5.556, 'HSE static time': 189.814, 'HSE optics time': 173.962, 'HSE few time': 15.988}


In [42]:
missing = []
for material in key_materials:
    try:
        fig = all_fields(material,data)
        fig.savefig(f'abs_plots/{material}.pdf')
    except:
        print(f"Missing data for {material}")
        missing.append(material)

Following entries altered for GaAs in dictionary {'GGA gap': 0.42019999999999946, 'HSE static gap': 1.3794999999999997, 'HSE few gap': 1.6846000000000005, 'GGA + corr SLME': 29.40895389040265, 'HSE SLME': 25.01346913470742, 'GGA static time': 21.555, 'GGA bands time': 98.181, 'GGA optics time': 16.543, 'HSE static time': 545.494, 'HSE optics time': 702.546, 'HSE few time': 47.27}
Following entries altered for InP in dictionary {'GGA gap': 0.4942000000000002, 'HSE static gap': 1.3791000000000002, 'HSE few gap': 2.684, 'GGA + corr SLME': 25.418717082083507, 'HSE SLME': 23.03413934898326, 'GGA static time': 12.566, 'GGA bands time': 5.735, 'GGA optics time': 7.056, 'HSE static time': 231.889, 'HSE optics time': 213.787, 'HSE few time': 69.936}
Following entries altered for CdTe in dictionary {'GGA gap': 0.5479999999999998, 'HSE static gap': 1.4345, 'HSE few gap': 1.9522, 'GGA + corr SLME': 24.910146037969067, 'HSE SLME': 22.703353067513564, 'GGA static time': 13.908, 'GGA bands time': 6.5

In [43]:
for material in key_materials:
    if material in missing:
        try:
            fig = all_fields(material,data)
            fig.savefig(f'abs_plots/{material}.pdf')
            missing.remove(material)
        except:
            print(f"Missing data for {material}")

In [44]:
missing

[]

In [45]:
data["InP"]

{'GGA gap': 0.4942000000000002,
 'HSE static gap': 1.3791000000000002,
 'HSE few gap': 2.684,
 'GGA + corr SLME': 25.418717082083507,
 'HSE SLME': 23.03413934898326,
 'GGA static time': 12.566,
 'GGA bands time': 5.735,
 'GGA optics time': 7.056,
 'HSE static time': 231.889,
 'HSE optics time': 213.787,
 'HSE few time': 69.936}

In [46]:
headers = list(data["InP"].keys())

In [47]:
df = pd.DataFrame(data).T

In [48]:
df

,GGA gap,HSE static gap,HSE few gap,GGA + corr SLME,HSE SLME,GGA static time,GGA bands time,GGA optics time,HSE static time,HSE optics time,HSE few time
GaAs,0.4202,1.3795,1.6846,29.408954,25.013469,21.555,98.181,16.543,545.494,702.546,47.270
InP,0.4942,1.3791,2.6840,25.418717,23.034139,12.566,5.735,7.056,231.889,213.787,69.936
CdTe,0.5480,1.4345,1.9522,24.910146,22.703353,13.908,6.577,7.789,252.441,190.582,53.780
MAPbI3,1.6834,2.3827,2.3830,11.455984,13.881105,121.952,130.802,198.756,596.506,505.214,136.774
GaCuSe2,0.1252,1.3668,1.7600,27.734243,22.006432,29.675,20.594,35.820,749.109,635.948,39.792
InCuSe2,0.0095,0.7630,1.1702,10.375573,23.560902,33.098,26.208,41.523,786.352,624.712,40.261
ZnCu2SnS4,0.1509,1.3468,1.7550,28.697312,23.637190,32.064,24.935,50.326,4583.033,3878.417,38.187
CuSbS2,0.7096,1.6112,1.5342,25.693751,26.452798,49.224,43.890,105.457,5219.081,2842.610,170.495
Sb2S3,1.2561,1.8564,1.7479,22.325574,19.452758,78.297,42.645,85.909,14583.390,12362.016,428.321
Cu2O,0.5258,2.0653,2.3741,5.555936,7.780332,13.429,7.060,13.238,378.456,212.456,94.180


In [49]:
avg_row = df.mean(axis=0)

In [55]:
avg_row[5:]

GGA static time      41.333923
GGA bands time       37.169385
GGA optics time      55.148615
HSE static time    2546.362308
HSE optics time    2032.267769
HSE few time        128.493462
dtype: float64

In [56]:
calcs = ["GGA static", "GGA bands", "GGA optics", "HSE static", "HSE optics", "HSE few"]

In [60]:
times_dict = {time:calc for calc,time in zip(avg_row[5:],calcs)}

In [61]:
times_dict

{'GGA static': 41.33392307692308,
 'GGA bands': 37.16938461538461,
 'GGA optics': 55.148615384615404,
 'HSE static': 2546.3623076923072,
 'HSE optics': 2032.2677692307693,
 'HSE few': 128.49346153846156}

In [63]:
dumpjson(times_dict,'Calc_times.json')